# Kirana Store Management System (Full Notebook)
**Author:** Bhuvan Gupta

This notebook is a complete, runnable end-to-end project using **SQLite** and **Python** (pandas). It includes:
- Correct, tested SQL schema (SQLite) in `kirana_schema_fixed.sql`.
- Functions to add/update inventory, record sales, and generate daily sales reports.
- Example usage and tests.


In [ ]:
import sqlite3
import pandas as pd
from datetime import datetime, date
import os
import json

DB_FILE = 'kirana_store.db'  # SQLite DB file created in the current working directory
SCHEMA_SQL = 'kirana_schema_fixed.sql'  # SQL file included in project

def get_conn(db_file=DB_FILE):
    conn = sqlite3.connect(db_file, detect_types=sqlite3.PARSE_DECLTYPES|sqlite3.PARSE_COLNAMES)
    conn.execute('PRAGMA foreign_keys = ON;')
    return conn


In [ ]:
def init_db(use_sql_file=True, db_file=DB_FILE, sql_path=SCHEMA_SQL):
    """Initialize the SQLite database. If use_sql_file and sql_path exists, it will run that SQL.
    Otherwise it will create tables programmatically with the same schema."""
    if use_sql_file and os.path.exists(sql_path):
        with open(sql_path, 'r') as f:
            sql = f.read()
        conn = get_conn(db_file)
        conn.executescript(sql)
        conn.commit()
        conn.close()
        print(f'Database created from SQL file: {sql_path}')
        return
    # Programmatic fallback (same schema)
    conn = get_conn(db_file)
    cur = conn.cursor()
    cur.execute('''
    CREATE TABLE IF NOT EXISTS products (
        product_id INTEGER PRIMARY KEY AUTOINCREMENT,
        name TEXT NOT NULL UNIQUE,
        category TEXT,
        quantity INTEGER NOT NULL DEFAULT 0,
        cost_price REAL NOT NULL DEFAULT 0.0,
        selling_price REAL NOT NULL DEFAULT 0.0
    )
    ''')
    cur.execute('''
    CREATE TABLE IF NOT EXISTS sales (
        sale_id INTEGER PRIMARY KEY AUTOINCREMENT,
        sale_datetime TEXT NOT NULL DEFAULT (datetime('now'))
    )
    ''')
    cur.execute('''
    CREATE TABLE IF NOT EXISTS sale_items (
        sale_item_id INTEGER PRIMARY KEY AUTOINCREMENT,
        sale_id INTEGER NOT NULL,
        product_id INTEGER NOT NULL,
        quantity INTEGER NOT NULL,
        unit_price REAL NOT NULL,
        FOREIGN KEY (sale_id) REFERENCES sales(sale_id) ON DELETE CASCADE,
        FOREIGN KEY (product_id) REFERENCES products(product_id)
    )
    ''')
    conn.commit()
    conn.close()
    print('Database initialized programmatically.')


In [ ]:
def add_or_update_product(name, category, quantity, cost_price, selling_price, db_file=DB_FILE):
    """Add a new product or update existing product (by name).
    If product exists, quantity is increased and prices updated to provided values."""
    if quantity < 0:
        raise ValueError('Quantity must be non-negative')
    conn = get_conn(db_file)
    cur = conn.cursor()
    try:
        cur.execute('SELECT product_id, quantity FROM products WHERE name = ?', (name,))
        row = cur.fetchone()
        if row:
            pid, existing_qty = row
            new_qty = existing_qty + int(quantity)
            cur.execute('UPDATE products SET quantity = ?, cost_price = ?, selling_price = ? WHERE product_id = ?',
                        (new_qty, float(cost_price), float(selling_price), pid))
            conn.commit()
            return pid
        else:
            cur.execute('INSERT INTO products (name, category, quantity, cost_price, selling_price) VALUES (?, ?, ?, ?, ?)',
                        (name, category, int(quantity), float(cost_price), float(selling_price)))
            conn.commit()
            return cur.lastrowid
    finally:
        conn.close()


In [ ]:
def get_all_products(db_file=DB_FILE):
    conn = get_conn(db_file)
    df = pd.read_sql_query('SELECT * FROM products ORDER BY name', conn)
    conn.close()
    return df


In [ ]:
def record_sale(items, db_file=DB_FILE):
    """
    Record a sale.
    items: list of dicts [{'product_id': int, 'quantity': int, 'unit_price': float}, ...]
    Ensures stock availability, reduces stock, and inserts sale and sale_items atomically.
    Returns sale_id on success.
    """
    if not items:
        raise ValueError('No items provided for sale')
    conn = get_conn(db_file)
    cur = conn.cursor()
    try:
        # Validate stock first
        for it in items:
            pid = int(it['product_id'])
            qty = int(it['quantity'])
            if qty <= 0:
                raise ValueError(f'Invalid quantity for product {pid}: {qty}')
            cur.execute('SELECT quantity FROM products WHERE product_id = ?', (pid,))
            r = cur.fetchone()
            if not r:
                raise ValueError(f'Product id {pid} not found')
            if r[0] < qty:
                raise ValueError(f'Not enough stock for product_id {pid}: requested {qty}, available {r[0]}')
        # All validations passed; insert sale
        cur.execute('INSERT INTO sales (sale_datetime) VALUES (?)', (datetime.now().isoformat(),))
        sale_id = cur.lastrowid
        for it in items:
            pid = int(it['product_id'])
            qty = int(it['quantity'])
            unit_price = float(it['unit_price'])
            cur.execute('INSERT INTO sale_items (sale_id, product_id, quantity, unit_price) VALUES (?, ?, ?, ?)',
                        (sale_id, pid, qty, unit_price))
            cur.execute('UPDATE products SET quantity = quantity - ? WHERE product_id = ?', (qty, pid))
        conn.commit()
        return sale_id
    except Exception:
        conn.rollback()
        raise
    finally:
        conn.close()


In [ ]:
def daily_sales_report(report_date=None, db_file=DB_FILE):
    """
    report_date: 'YYYY-MM-DD' string or None for today.
    Returns pandas DataFrame with columns: product_id, name, qty_sold, revenue, cost, profit
    """
    if report_date is None:
        report_date = date.today().isoformat()
    conn = get_conn(db_file)
    sql = '''
    SELECT p.product_id, p.name,
           SUM(si.quantity) AS qty_sold,
           SUM(si.quantity * si.unit_price) AS revenue,
           SUM(si.quantity * p.cost_price) AS cost,
           SUM(si.quantity * si.unit_price) - SUM(si.quantity * p.cost_price) AS profit
    FROM sale_items si
    JOIN sales s ON s.sale_id = si.sale_id
    JOIN products p ON p.product_id = si.product_id
    WHERE date(s.sale_datetime) = ?
    GROUP BY p.product_id, p.name
    ORDER BY qty_sold DESC
    '''
    df = pd.read_sql_query(sql, conn, params=(report_date,))
    conn.close()
    if df.empty:
        return df
    totals = pd.DataFrame([{ 'product_id': 'TOTAL', 'name': 'TOTAL',
                            'qty_sold': int(df['qty_sold'].sum()),
                            'revenue': float(df['revenue'].sum()),
                            'cost': float(df['cost'].sum()),
                            'profit': float(df['profit'].sum()) }])
    df = pd.concat([df, totals], ignore_index=True)
    return df


In [ ]:
# --- Demo: initialize DB from SQL, show products, record a sale, and show report ---
init_db(use_sql_file=True)
print('\nCurrent products:')
print(get_all_products().to_string(index=False))

# Add a new product
pid_rice = add_or_update_product('Rice (1kg)', 'Staples', 30, 40.0, 50.0)
pid_sugar = add_or_update_product('Sugar (1kg)', 'Staples', 20, 30.0, 40.0)
print(f'Added/Updated products with ids: {pid_rice}, {pid_sugar}')
print('\nProducts after adding:')
print(get_all_products().to_string(index=False))

# Record a sale (safe stock validation)
items_to_sell = [
    { 'product_id': pid_sugar, 'quantity': 2, 'unit_price': 40.0 },
    { 'product_id': pid_rice, 'quantity': 1, 'unit_price': 50.0 }
]
sale_id = record_sale(items_to_sell)
print(f'Created sale id: {sale_id}')

print('\nProducts after sale (stock reduced):')
print(get_all_products().to_string(index=False))

print('\nToday\'s sales report:')
display(daily_sales_report())


## Notes
- This notebook is tested against SQLite. If you want to run against MySQL/Postgres, adapt the SQL (AUTO_INCREMENT, SERIAL) and use SQLAlchemy for connections.
- Profit uses current `cost_price` stored in products. For historical accuracy, store cost per sale item.
- You can extend with returns, discounts, taxes, and export reports to CSV/Excel.
